In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np

import pandas as pd
import glob
import matplotlib.pyplot as plt

In [6]:
inputs = tf.random.normal([32,10,8])
gru = tf.keras.layers.GRU(4)
output = gru(inputs)
print(output)

tf.Tensor(
[[-0.23781747 -0.15315486 -0.34899867  0.12209348]
 [-0.54740393  0.42288393  0.02023208  0.6886072 ]
 [ 0.5177108   0.04065516 -0.01037304 -0.17039546]
 [ 0.07029843 -0.27201062 -0.45512447 -0.46180692]
 [ 0.25901076 -0.63453823  0.2925306  -0.26584962]
 [ 0.14051133 -0.4874902  -0.22394475 -0.27540037]
 [ 0.15024444 -0.51300013 -0.6037373   0.12581985]
 [ 0.43074265  0.46191797  0.19692297  0.8495231 ]
 [ 0.34717202  0.38504228 -0.0416719   0.01614207]
 [-0.21867912 -0.42364112 -0.11132351  0.28390002]
 [ 0.28751072 -0.09755865 -0.04072569  0.07671269]
 [ 0.4587091   0.22972907  0.04655197  0.5703226 ]
 [ 0.28103486  0.04164924 -0.2691552   0.43319216]
 [ 0.6479669  -0.58437616  0.65200394 -0.04018712]
 [-0.54791355  0.8114711  -0.49410695  0.9436602 ]
 [ 0.22608277 -0.32645634  0.24371397 -0.48655194]
 [ 0.08535485  0.36760992  0.21069336  0.8328542 ]
 [-0.5150851   0.5728459  -0.4814723   0.26637113]
 [-0.14588977  0.175538    0.1335554   0.27692604]
 [-0.25922054  0.482

In [7]:
gru = tf.keras.layers.GRU(4, return_sequences=True, return_state=True)
whole_sequence_output, final_state = gru(inputs)
print(whole_sequence_output.shape)

(32, 10, 4)


# 先行研究

## 発話エンコーダ設定

In [12]:
utter_embedding = tf.keras.layers.Embedding(1000, 300)
utter_gru = tf.keras.layers.GRU(512)

## 文脈エンコーダ設定

In [13]:
context_gru = tf.keras.layers.GRU(513)

## 対話行為エンコーダ設定

In [15]:
dialog_embedding = tf.keras.layers.Embedding(1000, 100)
dialog_gru = tf.keras.layers.GRU(128)

## 分類器設定

## 損失関数設定

## 最適化関数設定

## 実行

### filepath

In [3]:
train_data = "../Switchboard-Corpus/swda_data/train_set.txt"
test_data = "../Switchboard-Corpus/swda_data/test_set.txt"
val_data = "../Switchboard-Corpus/swda_data/val_set.txt"

In [5]:
train_dataset = []
f = open(train_data, "r", encoding='utf-8')
for row in f:
    train_dataset.append(row.strip())
f.close()

In [6]:
utter_user = []
utter = []
utter_label = []

for j in train_dataset:
    for i,v in enumerate(j.split("|")):
        if i == 0:
            utter_user.append(v)
        elif i==1:
            utter.append(v)
        else:
            utter_label.append(v)
print("finish")

finish


In [8]:
utter_train_dataset = tf.data.Dataset.from_tensor_slices((utter, utter_label))

vocabulary_set = set()
tokenizer = tfds.features.text.Tokenizer()
for text_tensor, _ in utter_train_dataset:
    some_tokens = tokenizer.tokenize(text_tensor.numpy())
    vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)

encoder = tfds.features.text.TokenTextEncoder(vocabulary_set, tokenizer=tokenizer)

In [9]:
def encode(token, label):
    token = encoder.encode(token.numpy())
    label = np.array([label])
    return token, label

def tf_encoder(utter, label):
    return tf.py_function(encode, [utter, label],[tf.int64, tf.int64])

MAX_LENGTH = 40
def filter_max_length(x,y,max_length=MAX_LENGTH):
    return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)

In [10]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [11]:
train_data_set = utter_train_dataset.map(tf_encoder)
train_data_set = train_data_set.filter(filter_max_length)
train_data_set = train_data_set.cache()
train_data_set = train_data_set.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([-1], [-1]))
train_data_set.prefetch(tf.data.experimental.AUTOTUNE)

<PrefetchDataset shapes: ((None, None), (None, None)), types: (tf.int64, tf.int64)>

In [15]:
for i,v in train_data_set.take(1):
    print(i)
    print(v)

InvalidArgumentError: ValueError: setting an array element with a sequence.
Traceback (most recent call last):

  File "/opt/conda/lib/python3.8/site-packages/tensorflow/python/framework/ops.py", line 972, in __int__
    return int(self._numpy())

ValueError: invalid literal for int() with base 10: b'fo_o_fw_"_by_bc'


The above exception was the direct cause of the following exception:


Traceback (most recent call last):

  File "/opt/conda/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 242, in __call__
    return func(device, token, args)

  File "/opt/conda/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 140, in __call__
    outputs = [

  File "/opt/conda/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 141, in <listcomp>
    _maybe_copy_to_context_device(self._convert(x, dtype=dtype),

  File "/opt/conda/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 120, in _convert
    return ops.conver

In [16]:
train_data_set.take(1)

<TakeDataset shapes: ((None, None), (None, None)), types: (tf.int64, tf.int64)>

### ウィンドウサイズ

In [21]:
window_size = 5

### 実行モデル